In [1]:
import os
from dotenv import load_dotenv

# Loading the .env file. Using full path to use it in the interactive shell
load_dotenv("C:\\Users\\MarcusChiri\\Dropbox\\Repositories\\Langchain\\.env")

# Loarding the environment variables
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BQpqxNch1D8mEoiVHh9YdLPOSL6kX', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff03006f-40f6-4570-9081-293230ad9df1-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don’t have that information. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BQpqzSbl5KTfxCoMyYBCznGwe7LJR', 'finish_reason': 'stop', 'logprobs': None}, id='run-a09ac8c0-b0ca-4f50-8cac-a35a4adcdfb3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 19, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 33, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BQpr2jqE2nn1Uznq4lx5AYRPENPsA', 'finish_reason': 'stop', 'logprobs': None}, id='run-30aa3178-97fa-413a-919b-ece1074a6e6d-0', usage_metadata={'input_tokens': 33, 'output_tokens': 13, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Message persistence
LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below. See its [documentation](https://langchain-ai.github.io/langgraph/concepts/persistence/) for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a thread_id. This should look like:

In [7]:
config = {"configurable": {"thread_id": "a1"}}

This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

We can then invoke the application:

In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [ ]:
for each in memory.list(config):
    print(*each, sep="\n")  # list all messages in memory
    print(each[1].get("channel_values"))
    break

next(x for x in memory.list(config))

{'configurable': {'thread_id': 'a1', 'checkpoint_ns': '', 'checkpoint_id': '1f023317-5542-6fa0-8001-dedf0b7d346a'}}
{'v': 3, 'ts': '2025-04-27T06:32:57.730652+00:00', 'id': '1f023317-5542-6fa0-8001-dedf0b7d346a', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.3259977664389192', 'messages': '00000000000000000000000000000003.0.1249272901216768', 'branch:to:model': '00000000000000000000000000000003.0.041495626647165995'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.1112195082354932'}, 'model': {'branch:to:model': '00000000000000000000000000000002.0.695414312139109'}}, 'channel_values': {'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f9f561a8-7225-4e2e-9c52-e4c14a617610'), AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23,

CheckpointTuple(config={'configurable': {'thread_id': 'a1', 'checkpoint_ns': '', 'checkpoint_id': '1f023317-5542-6fa0-8001-dedf0b7d346a'}}, checkpoint={'v': 3, 'ts': '2025-04-27T06:32:57.730652+00:00', 'id': '1f023317-5542-6fa0-8001-dedf0b7d346a', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.3259977664389192', 'messages': '00000000000000000000000000000003.0.1249272901216768', 'branch:to:model': '00000000000000000000000000000003.0.041495626647165995'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.1112195082354932'}, 'model': {'branch:to:model': '00000000000000000000000000000002.0.695414312139109'}}, 'channel_values': {'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f9f561a8-7225-4e2e-9c52-e4c14a617610'), AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prom

In [31]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today?


In [ ]:
for each in memory.list(config):
    print(*each, sep="\n")  # list all messages in memory
    print(each[1].get("channel_values"))
    break

next(x[1].get("channel_values")
     for i, x in enumerate(memory.list(config)) if i == 0)

{'configurable': {'thread_id': 'a1', 'checkpoint_ns': '', 'checkpoint_id': '1f02332f-ce89-6e27-8004-579eb0153823'}}
{'v': 3, 'ts': '2025-04-27T06:43:54.692560+00:00', 'id': '1f02332f-ce89-6e27-8004-579eb0153823', 'channel_versions': {'__start__': '00000000000000000000000000000005.0.3994628964424557', 'messages': '00000000000000000000000000000006.0.8835049635854708', 'branch:to:model': '00000000000000000000000000000006.0.15419316571915231'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000004.0.9916471233621325'}, 'model': {'branch:to:model': '00000000000000000000000000000005.0.3044396008913688'}}, 'channel_values': {'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f9f561a8-7225-4e2e-9c52-e4c14a617610'), AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23,

{'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f9f561a8-7225-4e2e-9c52-e4c14a617610'),
  AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BQprOPcoUlhnLl5nCb4c4JvI7Ec8q', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9f3afe1-ab82-444b-ae82-754495e52431-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="What's my name?", additiona

In [ ]:
config = {"configurable": {"thread_id": "a1"}}
query = "Sorry, what's my name again?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Your name is Bob! Is there something specific you would like to discuss, Bob?


In [ ]:
output  # output contains all messages in state

{'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f9f561a8-7225-4e2e-9c52-e4c14a617610'),
  AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BQprOPcoUlhnLl5nCb4c4JvI7Ec8q', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9f3afe1-ab82-444b-ae82-754495e52431-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="What's my name?", additiona

Great! Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [39]:
config = {"configurable": {"thread_id": "a2"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless it's shared with me in the course of our conversation. What would you like me to call you?


## Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

To add in a system message, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.

In [43]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

We can now update our application to incorporate this template:

In [44]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [45]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What be bringin' ye to these waters today? Arrr!


Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:

In [46]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Note that we have added a new language input to the prompt. Our application now has two parameters-- the input messages and language. We should update our application's state to reflect this:

In [47]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [52]:
config = {"configurable": {"thread_id": "b1"}}
query = "What is my favorite food?"
language = "English"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your favorite food is rabada, which is a dish made with beef tail. It's a flavorful and comforting meal! Do you have a favorite way to prepare or enjoy it?


Note that the entire state is persisted, so we can __omit parameters__ like language if no changes are desired:

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your favorite food is rabada.


## Managing Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

We can do this by adding a simple step in front of the prompt that modifies the messages key appropriately, and then wrap that new chain in the Message History class.

LangChain comes with a few built-in helpers for managing a list of messages. In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [57]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.

In [58]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history:

In [60]:
config = {"configurable": {"thread_id": "a2"}}
query = "What math question did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


## Streaming
Now we've got a functioning chatbot. However, one really important UX consideration for chatbot applications is streaming. LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most applications do is stream back each token as it is generated. This allows the user to see progress.

It's actually super easy to do this!

By default, .stream in our LangGraph application streams application steps-- in this case, the single step of the model response. Setting stream_mode="messages" allows us to stream output tokens instead:

In [65]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Please rewrite our whole conversation."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Sure|!| Here|’s| the| rewritten| conversation|:

|---

|**|Todd|:**| Hi|,| I'm| Todd|.| Please| tell| me| yet| another| joke|.

|**|Assistant|:**| Hi| Todd|!| Here|’s| another| joke| for| you|:| What| do| you| call| fake| spaghetti|?| An| imp|asta|!

|---| 

|Let| me| know| if| there's| anything| else| you'd| like|!||